**20L-0961 Quiz 02**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f3d71ff246db681043d9cbcaf4e39f70d3b54e752663fa08b87fab9a75958e53
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Quiz2").getOrCreate()
spark

# spark.stop()

In [3]:
rdd1 = spark.read.csv("emp1.txt", inferSchema=True, header=True)
rdd2 = spark.read.csv("emp2.txt", inferSchema=True, header=True)


# Show the RDDs
rdd1.show()
rdd2.show()

+------+--------+--------------------+
|emp_id|emp_name|            emp_dept|
+------+--------+--------------------+
|     1|    John|   Digital Marketing|
|     2|   Alice|Software Development|
|     3|     Bob|        Data Science|
|     4| Charlie|     Human Resources|
|     5|     Eve|             Finance|
|     6| Michael|Software Development|
|     7|  Olivia|        Data Science|
|     8|  Sophia|             Finance|
|     9|    Liam|        Data Science|
|    10|   Emily|Software Development|
|    11|   James|             Finance|
|    12|Benjamin|   Digital Marketing|
|    13|    Emma|Software Development|
|    14|     Ava|        Data Science|
|    15|Isabella|     Human Resources|
|    16|   Lucas|             Finance|
|    17|   Avery|Software Development|
|    18|   Grace|        Data Science|
|    19|   Ethan|     Human Resources|
|    20|Scarlett|Software Development|
+------+--------+--------------------+
only showing top 20 rows

+------+----------+--------------+
|em

In [4]:
# Inner Join: Perform an inner join on emp_id between rdd1 and rdd2 to get an RDD containing
# the employee ID, employee name, employee department, and employee salary for
# employees who exist inboth RDDs

result = rdd1.join(rdd2, rdd1.emp_id == rdd2.emp_id, "inner")
result.show()

+------+--------+--------------------+------+----------+--------------+
|emp_id|emp_name|            emp_dept|emp_id|emp_salary|emp_experience|
+------+--------+--------------------+------+----------+--------------+
|     1|    John|   Digital Marketing|     1|     50000|             2|
|     2|   Alice|Software Development|     2|     65000|             5|
|     3|     Bob|        Data Science|     3|     70000|             3|
|     4| Charlie|     Human Resources|     4|     55000|             1|
|     5|     Eve|             Finance|     5|     60000|             4|
|     6| Michael|Software Development|     6|     80000|             6|
|     7|  Olivia|        Data Science|     7|     90000|             3|
|     8|  Sophia|             Finance|     8|     75000|             2|
|     9|    Liam|        Data Science|     9|     55000|             4|
|    10|   Emily|Software Development|    10|     70000|             5|
|    11|   James|             Finance|    11|     60000|        

In [5]:
# Left Outer Join: Perform a left outer join on emp_id between rdd1 and rdd2 to get an RDD
# containingthe employee ID, employee name, employee department, and employee salary
# for all employees in rdd1 and matching employees in rdd2. For employees in rdd1 that do
# not have a matching employee ID in rdd2, the employee salary should be set to None.

result = rdd1.join(rdd2, rdd1.emp_id == rdd2.emp_id, "left")
result.show()

+------+--------+--------------------+------+----------+--------------+
|emp_id|emp_name|            emp_dept|emp_id|emp_salary|emp_experience|
+------+--------+--------------------+------+----------+--------------+
|     1|    John|   Digital Marketing|     1|     50000|             2|
|     2|   Alice|Software Development|     2|     65000|             5|
|     3|     Bob|        Data Science|     3|     70000|             3|
|     4| Charlie|     Human Resources|     4|     55000|             1|
|     5|     Eve|             Finance|     5|     60000|             4|
|     6| Michael|Software Development|     6|     80000|             6|
|     7|  Olivia|        Data Science|     7|     90000|             3|
|     8|  Sophia|             Finance|     8|     75000|             2|
|     9|    Liam|        Data Science|     9|     55000|             4|
|    10|   Emily|Software Development|    10|     70000|             5|
|    11|   James|             Finance|    11|     60000|        

In [6]:
# Calculate the total salary of employees in RDD1 by joining it with RDD2 on
# "emp_id" andmultiplying "emp_salary" with "emp_experience".

result = rdd1.join(rdd2, rdd1.emp_id == rdd2.emp_id, "inner")
result = result.withColumn("total_salary", result.emp_salary * result.emp_experience)
result.show()

+------+--------+--------------------+------+----------+--------------+------------+
|emp_id|emp_name|            emp_dept|emp_id|emp_salary|emp_experience|total_salary|
+------+--------+--------------------+------+----------+--------------+------------+
|     1|    John|   Digital Marketing|     1|     50000|             2|      100000|
|     2|   Alice|Software Development|     2|     65000|             5|      325000|
|     3|     Bob|        Data Science|     3|     70000|             3|      210000|
|     4| Charlie|     Human Resources|     4|     55000|             1|       55000|
|     5|     Eve|             Finance|     5|     60000|             4|      240000|
|     6| Michael|Software Development|     6|     80000|             6|      480000|
|     7|  Olivia|        Data Science|     7|     90000|             3|      270000|
|     8|  Sophia|             Finance|     8|     75000|             2|      150000|
|     9|    Liam|        Data Science|     9|     55000|         

In [7]:
from pyspark.sql import SparkSession
from pyspark.broadcast import Broadcast

# SparkSession
spark = SparkSession.builder.appName("broadcast_join").getOrCreate()

# Read the CSV files into DataFrames
rdd1 = spark.read.csv("emp1.txt", header=True, inferSchema=True)
rdd2 = spark.read.csv("emp2.txt", header=True, inferSchema=True)

# Broadcast the first RDD
broadcast_rdd1 = spark.sparkContext.broadcast(rdd1.rdd.map(lambda x: (x["emp_id"], x)).collectAsMap())

# Perform a broadcast join on the emp_id column
result = rdd2.rdd.map(lambda x: (x["emp_id"], x["emp_salary"], broadcast_rdd1.value.get(x["emp_id"], {"emp_name": None, "emp_dept": None})["emp_name"], broadcast_rdd1.value.get(x["emp_id"], {"emp_name": None, "emp_dept": None})["emp_dept"])).toDF(["emp_id", "emp_salary", "emp_name", "emp_dept"])

# Display the result
result.show()

+------+----------+--------+--------------------+
|emp_id|emp_salary|emp_name|            emp_dept|
+------+----------+--------+--------------------+
|     1|     50000|    John|   Digital Marketing|
|     2|     65000|   Alice|Software Development|
|     3|     70000|     Bob|        Data Science|
|     4|     55000| Charlie|     Human Resources|
|     5|     60000|     Eve|             Finance|
|     6|     80000| Michael|Software Development|
|     7|     90000|  Olivia|        Data Science|
|     8|     75000|  Sophia|             Finance|
|     9|     55000|    Liam|        Data Science|
|    10|     70000|   Emily|Software Development|
|    11|     60000|   James|             Finance|
|    12|     45000|Benjamin|   Digital Marketing|
|    13|     85000|    Emma|Software Development|
|    14|     75000|     Ava|        Data Science|
|    15|     70000|Isabella|     Human Resources|
|    16|     80000|   Lucas|             Finance|
|    17|     55000|   Avery|Software Development|


In [11]:
spark.stop()

In [12]:
from pyspark import SparkContext
sc = SparkContext("local", "Accumulator")
rdd = sc.parallelize(range(1, 11))

# Define accumulator
accumulator = sc.accumulator(0)
def update_acc(x):
    accumulator.add(x)
rdd.foreach(update_acc)

# Print final value of accumulator
print(f"\nSum of Values using Accumulator:{accumulator.value}")


Sum of Values using Accumulator:55
